###  population

In [1]:
#loading all library
library(tidyverse)
library(readr)
library(knitr)
library(skimr)
library(tidyr)
install.packages('naniar')
library(naniar)


Warning message in system("timedatectl", intern = TRUE):
“running command 'timedatectl' had status 1”
── Attaching packages ─────────────────────────────────────────────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.3.5     ✔ purrr   0.3.4
✔ tibble  3.1.5     ✔ dplyr   1.0.7
✔ tidyr   1.1.3     ✔ stringr 1.4.0
✔ readr   2.0.0     ✔ forcats 0.5.1

── Conflicts ────────────────────────────────────────────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

Installing package into ‘/home/sekiro/R/x86_64-pc-linux-gnu-library/3.6’
(as ‘lib’ is unspecified)


Attaching package: ‘naniar’


The following object is masked from ‘package:skimr’:

    n_complete




###  population clean up

In [3]:
population <- read.csv(file="population/population1.csv", skip = 1)
population <- population %>% slice(4, 8, 12, 16, 20, 24, 28, 32, 36, 40, 44, 48, 52, 56, 60, 64)
#write.csv(population, file = "population_1.csv") 

### change_rate clean up

In [16]:
#change_rate clean up
change_rate <- read.csv(file="population/change rate.csv", skip = 1)
change_rate <- change_rate %>% rename(year = X, Population_change = Population.change)
change_rate <- change_rate %>% slice(1:16)
#save as csv
#write.csv(change_rate, file = "change_rate_1.csv")

### migrant clean up

In [6]:
migrant <- read.csv(file="population/migrant1.csv", skip = 5)
migrant <- migrant[,-7]
migrant <- read.csv(file="population/migrant1.csv", skip = 5)
migrant <- migrant[,-7]
migrant <- migrant %>% rename(year = X., Residence_Permanent_Residence = Estimate,
                             Student_Permanent_Residence= Estimate.1,
                             Visitor_Permanent_Residence = Estimate.2,
                             Work_Permanent_Residence = Estimate.3,
                             New_Zealand_and_Australian_citizens = Estimate.4,
                             TOTAL = Estimate.6)
migrant <- migrant %>% slice(1:16)
#write.csv(migrant, file = "migrant_1.csv")

### population combine change_rate

In [7]:
population <- population %>% mutate(year = c(2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016, 2017,2018,2019,2020))
population_change_rate <- merge(population, change_rate)
population_change_rate <- population_change_rate %>% rename(total_population = Total)
#write.csv(population_change_rate, file = "population_change_rate.csv") 

ERROR: Error in as.data.frame(y): object 'change_rate' not found


### region_population clean up

In [20]:
region_population <- read.csv(file="population/region population.csv", skip = 1)
region_population <- region_population %>% rename(year = X)
region_population <- region_population %>% slice(1:16)
#select the city we want to Performance
region_population2 <- region_population %>% select(year, Northland.Region, Auckland.Region, Wellington.Region, Canterbury.Region) %>%
rename(Northland = Northland.Region, Auckland = Auckland.Region, Wellington = Wellington.Region, Canterbury = Canterbury.Region)
region_population3 <- gather(region_population2, city, Total.Pop, Northland:Canterbury)%>%
                        mutate(Pop.change = if_else(city == lag(city), Total.Pop - lag(Total.Pop), Total.Pop-Total.Pop),
                              Pop.change = na_if(Pop.change, 0),
                              city = if_else(grepl("Northland",city), 'Auckland', city)) %>%
                        group_by(year, city) %>% summarise(Total.Pop = sum(Total.Pop), Pop.change = sum(Pop.change))
write.csv(region_population3, "FINAL_population_by_city.csv") # Save csv file for each city by year              

`summarise()` has grouped output by 'year'. You can override using the `.groups` argument.



In [21]:
# select the city we want and rename column
region_population2 <- region_population %>% select(year, Northland.Region, Auckland.Region, Wellington.Region, Canterbury.Region) %>%
rename(Northland = Northland.Region, Auckland = Auckland.Region, Wellington = Wellington.Region, Canterbury = Canterbury.Region)

#Calculate the amount of population change per year
region_population3 <- gather(region_population2, city, Total.Pop, Northland:Canterbury)%>%
                        mutate(Pop.change = if_else(city == lag(city), Total.Pop - lag(Total.Pop), Total.Pop-Total.Pop),
                              Pop.change = na_if(Pop.change, 0),
                              city = if_else(grepl("Northland",city), 'Auckland', city)) %>%
                        group_by(year, city) %>% summarise(Total.Pop = sum(Total.Pop), Pop.change = sum(Pop.change))

# Save csv file for each city by year  
write.csv(region_population3, "FINAL_population_by_city.csv")             

`summarise()` has grouped output by 'year'. You can override using the `.groups` argument.

